# Generate 1D Chain JSON Data from Numpy Files

This notebook extracts real experimental data from numpy files and creates a JSON file for the dashboard.

In [3]:
import numpy as np

In [4]:
data = {"5q": {}, "100q": {}}
nq = 5
case = ""
prop = "r"

print("Processing 5-qubit data...\n")

Processing 5-qubit data...



In [5]:
# Define backends for 5q
backends_5q = ["qasm_simulator", "iqm_emerald", "iqm_garnet", "ibm_fez", 
               "ibm_marrakesh", "ibm_brisbane", "rigetti_ankaa_2", "rigetti_ankaa_3"]

for backend in backends_5q:
    try:
        results = np.load(f"./Data/{backend}/{nq}_1D.npy", allow_pickle=True).item()
        
        # Override iqm_emerald with AWS version
        if backend == "iqm_emerald":
            results = np.load(f"./Data/iqm_emerald/{nq}_1D_aws.npy", allow_pickle=True).item()
        
        delta = results["Deltas"][0]
        ps = results["ps"]
        
        # Find best section based on highest mean
        best_mean = 0
        best_sec = None
        for sec_i in results[f"postprocessing{case}"][delta][ps[0]].keys():
            res_i = [results[f"postprocessing{case}"][delta][p][sec_i][prop] for p in ps]
            if np.mean(res_i) > best_mean:
                best_sec = res_i
                best_mean = np.mean(res_i)
        
        # Extract random baseline
        random_r = results.get(f"random{case}", {}).get(prop, 0.667)
        
        data["5q"][backend] = {
            "qubits": nq,
            "p_values": ps,
            "r_values": [round(r, 3) for r in best_sec],
            "max_r": round(max(best_sec), 3),
            "optimal_p": int(ps[np.argmax(best_sec)]),
            "random_r": round(random_r, 3)
        }
        print(f"  {backend}: max_r = {round(max(best_sec), 3)} at p = {ps[np.argmax(best_sec)]}")
        
    except Exception as e:
        print(f"  Warning: Could not load {backend}: {e}")

  qasm_simulator: max_r = 1.0 at p = 50
  iqm_emerald: max_r = 0.838 at p = 5
  iqm_garnet: max_r = 0.831 at p = 5
  ibm_fez: max_r = 0.868 at p = 9
  ibm_marrakesh: max_r = 0.923 at p = 15
  ibm_brisbane: max_r = 0.843 at p = 5
  rigetti_ankaa_2: max_r = 0.608 at p = 3
  rigetti_ankaa_3: max_r = 0.604 at p = 3


In [6]:
# Add special cases for 5q
try:
    results = np.load(f"./Data/originq_wukong/{nq}_1D_2.npy", allow_pickle=True).item()
    delta = results["Deltas"][0]
    ps = results["ps"]
    best_mean = 0
    for sec_i in results[f"postprocessing{case}"][delta][ps[0]].keys():
        res_i = [results[f"postprocessing{case}"][delta][p][sec_i][prop] for p in ps]
        if np.mean(res_i) > best_mean:
            best_sec = res_i
            best_mean = np.mean(res_i)
    
    data["5q"]["originq_wukong"] = {
        "qubits": nq,
        "p_values": ps,
        "r_values": [round(r, 3) for r in best_sec],
        "max_r": round(max(best_sec), 3),
        "optimal_p": int(ps[np.argmax(best_sec)]),
        "random_r": 0.667
    }
    print(f"  originq_wukong: max_r = {round(max(best_sec), 3)} at p = {ps[np.argmax(best_sec)]}")
except Exception as e:
    print(f"  Warning: Could not load originq_wukong: {e}")

try:
    results = np.load(f"./Data/iqm_sirius/{nq}_1D_Single.npy", allow_pickle=True).item()
    delta = results["Deltas"][0]
    ps = results["ps"]
    best_mean = 0
    for sec_i in results[f"postprocessing{case}"][delta][ps[0]].keys():
        res_i = [results[f"postprocessing{case}"][delta][p][sec_i][prop] for p in ps]
        if np.mean(res_i) > best_mean:
            best_sec = res_i
            best_mean = np.mean(res_i)
    
    data["5q"]["iqm_sirius"] = {
        "qubits": nq,
        "p_values": ps,
        "r_values": [round(r, 3) for r in best_sec],
        "max_r": round(max(best_sec), 3),
        "optimal_p": int(ps[np.argmax(best_sec)]),
        "random_r": 0.667
    }
    print(f"  iqm_sirius: max_r = {round(max(best_sec), 3)} at p = {ps[np.argmax(best_sec)]}")
except Exception as e:
    print(f"  Warning: Could not load iqm_sirius: {e}")

  originq_wukong: max_r = 0.832 at p = 4
  iqm_sirius: max_r = 0.685 at p = 3


In [7]:
print(f"\nProcessing 100-qubit data...\n")
nq = 100
delta = 1
kk = 0

backends_100q = ["ibm_marrakesh", "ibm_brisbane", "ibm_sherbrooke", "ibm_kyiv", 
                 "ibm_nazca", "ibm_kyoto", "ibm_osaka", "ibm_fez", "ibm_brussels", 
                 "ibm_strasbourg"]

for backend in backends_100q:
    try:
        results = np.load(f"./Data/{backend}/{nq}_1D.npy", allow_pickle=True).item()
        res_backend = results[f"postprocessing{case}"]
        ps = list(res_backend[delta].keys())
        rs = [res_backend[delta][p][kk][prop] for p in ps]
        
        # Get random baseline
        if backend == "ibm_brisbane":
            random_r = results.get(f"random{case}", {}).get(prop, 0.50)
        else:
            random_r = 0.50
        
        data["100q"][backend] = {
            "qubits": nq,
            "p_values": ps,
            "r_values": [round(r, 3) for r in rs],
            "max_r": round(max(rs), 3),
            "optimal_p": int(ps[np.argmax(rs)]),
            "random_r": round(random_r, 3)
        }
        print(f"  {backend}: max_r = {round(max(rs), 3)} at p = {ps[np.argmax(rs)]}")
        
    except Exception as e:
        print(f"  Warning: Could not load {backend}: {e}")


Processing 100-qubit data...

  ibm_marrakesh: max_r = 0.773 at p = 8
  ibm_brisbane: max_r = 0.756 at p = 5
  ibm_sherbrooke: max_r = 0.721 at p = 5
  ibm_kyiv: max_r = 0.723 at p = 4
  ibm_nazca: max_r = 0.673 at p = 5
  ibm_kyoto: max_r = 0.662 at p = 4
  ibm_osaka: max_r = 0.675 at p = 8
  ibm_fez: max_r = 0.777 at p = 9
  ibm_brussels: max_r = 0.719 at p = 5
  ibm_strasbourg: max_r = 0.711 at p = 5


In [8]:
# Add torino variants
try:
    results = np.load(f"./Data/ibm_torino/{nq}_1D_v1.npy", allow_pickle=True).item()
    res_backend = results[f"postprocessing{case}"]
    ps = list(res_backend[delta].keys())
    rs = [res_backend[delta][p][kk][prop] for p in ps]
    
    data["100q"]["ibm_torino-v1"] = {
        "qubits": nq,
        "p_values": ps,
        "r_values": [round(r, 3) for r in rs],
        "max_r": round(max(rs), 3),
        "optimal_p": int(ps[np.argmax(rs)]),
        "random_r": 0.50
    }
    print(f"  ibm_torino-v1: max_r = {round(max(rs), 3)} at p = {ps[np.argmax(rs)]}")
except Exception as e:
    print(f"  Warning: Could not load ibm_torino-v1: {e}")

try:
    results = np.load(f"./Data/ibm_torino/{nq}_1D.npy", allow_pickle=True).item()
    res_backend = results[f"postprocessing{case}"]
    ps = list(res_backend[delta].keys())
    rs = [res_backend[delta][p][kk][prop] for p in ps]
    
    data["100q"]["ibm_torino-v0"] = {
        "qubits": nq,
        "p_values": ps,
        "r_values": [round(r, 3) for r in rs],
        "max_r": round(max(rs), 3),
        "optimal_p": int(ps[np.argmax(rs)]),
        "random_r": 0.50
    }
    print(f"  ibm_torino-v0: max_r = {round(max(rs), 3)} at p = {ps[np.argmax(rs)]}")
except Exception as e:
    print(f"  Warning: Could not load ibm_torino-v0: {e}")

try:
    results = np.load(f"./Data/ibm_boston/{nq}_1D.npy", allow_pickle=True).item()
    res_backend = results[f"postprocessing{case}"]
    ps = list(res_backend[delta].keys())
    rs = [res_backend[delta][p][kk][prop] for p in ps]
    
    data["100q"]["ibm_boston"] = {
        "qubits": nq,
        "p_values": ps,
        "r_values": [round(r, 3) for r in rs],
        "max_r": round(max(rs), 3),
        "optimal_p": int(ps[np.argmax(rs)]),
        "random_r": 0.50
    }
    print(f"  ibm_boston: max_r = {round(max(rs), 3)} at p = {ps[np.argmax(rs)]}")
except Exception as e:
    print(f"  Warning: Could not load ibm_boston: {e}")

  ibm_torino-v1: max_r = 0.76 at p = 9
  ibm_torino-v0: max_r = 0.728 at p = 9
  ibm_boston: max_r = 0.841 at p = 10


In [ ]:
# Save to JSON
output_path = Path("Data/1d_chain_processed.json")
with open(output_path, 'w') as f:
    json.dump(data, f, indent=2)

print(f"\n✓ Successfully saved data to {output_path}")
print(f"  5q backends: {len(data['5q'])}")
print(f"  100q backends: {len(data['100q'])}")